In [1]:
import pandas as pd
import re 
pd.options.mode.chained_assignment = None
df=pd.read_csv('20220906 competitor_final.csv')

In [2]:
similar_units = {'PCS': 'PC',
                 'PC': 'PC',
                 'PIECES':'PC',
                 'KG': 'KILOGRAM',
                 'KGS': 'KILOGRAM',
                 'GM':'GRAM',
                 'GMS':'GRAM',
                 'LTR':'LITRE',
                 'LITRES':'LITRE',
                 'LITER':'LITRE',
                 'WATTS':'WATT',
                 'FT':'FEET',
                 'LTE':'LITE',
                 'INCHES':'INCH',
                 '"':'INCH',
                 'FEETS':'FEET',
                 'B1G1':'BOGO',
                 'BUY 1 GET 1':'BOGO',
                 'BTU':'BTUS',
                 'TON':'TON',
                 'TN':'TON',
                 'MG':'ML',
                 'ML':'ML',
                 'SINGLE PACK': '1PC',
                 'SINGLE PIECE':'1PC',
                 'SINGLE PC':'1PC',
                 'SACHETS':'SACHET',
                 'STICKS':'STICK'
                 }
ignore_words = {'FREE','COMBO','BUNDLE','CASE',
'WARRANTY','GUARANTEE','SET',
'WHITE','BLACK','SILVER','BLUE','RED','GREEN','YELLOW','GREY',
'CARTON'}
regex_ignore = '\D+x\D+'

In [6]:
def name_processing(accuracy=0.7,reject_ignore_words=True):
    name_cols = ['PRODUCT_NAME','ITEM_NAME']
    modified_cols = ['DARAZ_PROD','COMP_PROD']
    brand_cols = ['DARAZ_BRAND','COMP_BRAND']
    # splitting into list of uppercased words
    df[modified_cols]= df[name_cols].apply(lambda x:x.str.upper())
    for word, initial in similar_units.items():
        df[modified_cols]= df[modified_cols].apply(lambda x:x.str.replace(word,initial))
    df[modified_cols]= df[modified_cols].apply(lambda x:x.str.split(' '))
    # removing ignore words
    if reject_ignore_words:
        df[modified_cols]= df[modified_cols].applymap(lambda x:[ele for ele in x if ele not in ignore_words])
    else:
        pass    
    # matching the first word as brand name
    df[brand_cols] = df[modified_cols].applymap(lambda x:x[0])
    bm = df[df['DARAZ_BRAND']==df['COMP_BRAND']]
    # taking count of matched words between two columns/companies
    bm['MATCHED_WORD_COUNT'] = [len(set(a).intersection(b)) for a, b in zip(bm['DARAZ_PROD'], bm['COMP_PROD'])]
    # taking the lowest count of word
    bm['LOW_C']= bm[modified_cols].applymap(len).min(axis=1)
    bm['MR'] = bm['MATCHED_WORD_COUNT']/bm['LOW_C']
    # setting the accuracy rate
    tg_list =bm[bm['MR']>accuracy]
    return tg_list.sort_values(['MR'])


In [9]:
name_processing(reject_ignore_words=True).to_clipboard()